[<font color = 'coral'>References Code</font>](https://github.com/vitaldb/examples/blob/master/eeg_mac.ipynb)

# Prediction of anesthetic concentration from EEG
In this example, we will build a model to predict anesthetic concentration (age-related MAC) from EEG during Sevoflurane anesthesia.

> Note that <b>all users who use Vital DB, an open biosignal dataset, must agree to the Data Use Agreement below.
</b> If you do not agree, please close this window.
Click here: [Data Use Agreement](https://vitaldb.net/dataset/?query=overview&documentId=13qqajnNZzkN7NZ9aXnaQ-47NWy7kx-a6gbrcEsi-gak&sectionId=h.vcpgs1yemdb5)

## Required libraries

In [2]:
!pip install vitaldb
import vitaldb
import random
import numpy as np
import pandas as pd
import scipy.signal
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 9.5 MB/s eta 0:00:00


## Preprocessing

In [3]:
SRATE = 128  # in hz
SEGLEN = 4 * SRATE  # segment samples
MAX_CASES = 100

df_trks = pd.read_csv("https://api.vitaldb.net/trks")  # track information
df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # patient information

# track names and column order when loading data
track_names = ['BIS/EEG1_WAV', 'BIS/BIS']


# Inclusion & Exclusion criteria
caseids = set(df_cases.loc[df_cases['age'] > 18, 'caseid'])
caseids &= set(df_trks.loc[df_trks['tname'] == 'BIS/EEG1_WAV', 'caseid'])
caseids &= set(df_trks.loc[df_trks['tname'] == 'BIS/BIS', 'caseid'])
caseids = list(caseids)
print(f'Total {len(caseids)} cases found')

Total 5800 cases found


## Filtering input data

invalid samples...

C:\Users\ASUS\AppData\Local\Temp\ipykernel_25392\2705416851.py:4: RuntimeWarning: All-NaN slice encountered
  valid_mask &= (np.nanmax(x, axis=1) - np.nanmin(x, axis=1) > 12)  # bis impedence check
C:\Users\ASUS\AppData\Local\Temp\ipykernel_25392\2705416851.py:5: RuntimeWarning: All-NaN slice encountered
  valid_mask &= (np.nanmax(np.abs(x), axis=1) < 100)  # noisy sample


12.7% removed


## Splitting samples into training and testing dataset

In [ ]:
# caseid
caseids = list(np.unique(c))
#random.shuffle(caseids)

# Split dataset into training and testing data
ntest = max(1, int(len(caseids) * 0.2))
caseids_train = caseids[ntest:]
caseids_test = caseids[:ntest]

train_mask = np.isin(c, caseids_train)
test_mask = np.isin(c, caseids_test)
x_train = x[train_mask]
y_train = y[train_mask]
x_test = x[test_mask]
y_test = y[test_mask]
b_test = b[test_mask]
c_test = c[test_mask]

print('====================================================')
print(f'total: {len(caseids)} cases {len(y)} samples')
print(f'train: {len(np.unique(c[train_mask]))} cases {len(y_train)} samples')
print(f'test {len(np.unique(c_test))} cases {len(y_test)} samples')
print('====================================================')

total: 100 cases 496346 samples
train: 80 cases 384419 samples
test 20 cases 111927 samples


In [ ]:
x_train.shape, y_train.shape

((384419, 512), (384419,))

## Modeling and Evaluation

#LSTM

In [ ]:
X = x_train
y = y_train

In [ ]:
import keras.models
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Input, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint

out = inp = Input(shape=(x_train.shape[1], 1))
for i in range(4):
    out = Conv1D(filters=32, kernel_size=7, padding='same')(out)
    out = Activation('relu')(out)
    out = MaxPooling1D(2, padding='same')(out)
out = GlobalMaxPooling1D()(out)
out = Dense(128)(out)
out = Dropout(0.2)(out)
out = Dense(1)(out)

model = Model(inputs=[inp], outputs=[out])
model.summary()
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])

callbacks = [
    ModelCheckpoint(monitor='val_accuracy',
                    filepath='model.x',
                    verbose=1,
                    save_best_only=True),
    EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
]

hist = model.fit(x_train, y_train, validation_split=0.2, epochs=3, batch_size=4096, callbacks=callbacks)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 512, 1)]          0         
                                                                 
 conv1d_4 (Conv1D)           (None, 512, 32)           256       
                                                                 
 activation_4 (Activation)   (None, 512, 32)           0         
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 256, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_5 (Conv1D)           (None, 256, 32)           7200      
                                                                 
 activation_5 (Activation)   (None, 256, 32)           0         
                                                           

INFO:tensorflow:Assets written to: model.x\assets


76/76 [==============================] - 98s 1s/step - loss: 0.5767 - accuracy: 0.0047 - val_loss: 0.3512 - val_accuracy: 0.0016
Epoch 2/3
76/76 [==============================] - ETA: 0s - loss: 0.2618 - accuracy: 4.6499e-04
Epoch 2: val_accuracy did not improve from 0.00164
76/76 [==============================] - 106s 1s/step - loss: 0.2618 - accuracy: 4.6499e-04 - val_loss: 0.3488 - val_accuracy: 0.0012
Epoch 3/3
14/76 [====>.........................] - ETA: 1:20 - loss: 0.2383 - accuracy: 0.0016

KeyboardInterrupt: 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Assuming x_train, y_train, x_test, y_test are your training and test data

# Reshape data for LSTM input (assuming x_train and x_test are 3D tensors)
# The input shape should be (number_of_samples, time_steps, features)
x_train_reshaped = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test_reshaped = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(x_train_reshaped.shape[1], 1)))
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

# Print a summary of the model architecture
model.summary()

# Fit the model
hist = model.fit(x_train_reshaped, y_train, validation_split=0.2, epochs=3, batch_size=4096,
                 callbacks=[tf.keras.callbacks.ModelCheckpoint('model_lstm.h5', monitor='val_loss', verbose=1, save_best_only=True),
                            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')])

# Make predictions
pred_test = model.predict(x_test_reshaped).flatten()

# Apply post-processing if needed
# ...

# Calculate the performance
test_mae = np.mean(np.abs(y_test - pred_test))
print(f'Test MAE: {test_mae}')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 50)                10400     
                                                                 
 dense_5 (Dense)             (None, 128)               6528      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 17057 (66.63 KB)
Trainable params: 17057 (66.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/3
76/76 [==============================] - ETA: 0s - loss: 0.3185 - mean_absolute_error: 0.3185
Epoch 1: val_loss improved from inf to 0.42138

E:\Conda\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


76/76 [==============================] - ETA: 0s - loss: 0.2651 - mean_absolute_error: 0.2651
Epoch 2: val_loss improved from 0.42138 to 0.41848, saving model to model_lstm.h5
76/76 [==============================] - 287s 4s/step - loss: 0.2651 - mean_absolute_error: 0.2651 - val_loss: 0.4185 - val_mean_absolute_error: 0.4185
Epoch 3/3
76/76 [==============================] - ETA: 0s - loss: 0.2582 - mean_absolute_error: 0.2582
Epoch 3: val_loss improved from 0.41848 to 0.41708, saving model to model_lstm.h5
3498/3498 [==============================] - 103s 29ms/step
Test MAE: 0.26340937388276275


In [ ]:
from sklearn.metrics import r2_score

# Assuming y_test contains the true BIS scores and pred_test contains the predicted scores
r2 = r2_score(y_test, pred_test)
print(f'R-squared: {r2}')


NameError: name 'y_test' is not defined

## Classification approach CNN


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Assuming x_train, y_train, x_test, y_test are your training and test data

# Convert BIS scores to classes (you need to define your own bins or thresholds)
num_classes = 5  # Adjust based on your specific classification setup
y_train_classes = np.digitize(y_train, bins=np.linspace(0, 100, num_classes))
y_test_classes = np.digitize(y_test, bins=np.linspace(0, 100, num_classes))

# Reshape data for CNN input (assuming x_train and x_test are 3D tensors)
x_train_reshaped = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test_reshaped = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# Define the CNN model for classification
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=7, activation='relu', input_shape=(x_train_reshaped.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model for classification
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

# Fit the model for classification
hist = model.fit(x_train_reshaped, y_train_classes, validation_split=0.2, epochs=3, batch_size=4096,
                 callbacks=[tf.keras.callbacks.ModelCheckpoint('model_cnn_classification.h5', monitor='val_loss', verbose=1, save_best_only=True),
                            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')])

# Evaluate the model on the test set using accuracy
test_results = model.evaluate(x_test_reshaped, y_test_classes)
print(f'Test Accuracy: {test_results[1]}')


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 506, 32)           256       
                                                                 
 max_pooling1d_8 (MaxPoolin  (None, 253, 32)           0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 8096)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               1036416   
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 5)                 645       
                                                      

E:\Conda\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


76/76 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2: val_loss did not improve from 0.00000
76/76 [==============================] - 44s 578ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/3
76/76 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3: val_loss did not improve from 0.00000
76/76 [==============================] - 43s 562ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3: early stopping
3498/3498 [==============================] - 12s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test Accuracy: 1.0


# LSTM non regression

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Assuming x_train, y_train, x_test, y_test are your training and test data

# Convert BIS scores to classes (you need to define your own bins or thresholds)
num_classes = 5  # Adjust based on your specific classification setup
y_train_classes = np.digitize(y_train, bins=np.linspace(0, 100, num_classes))
y_test_classes = np.digitize(y_test, bins=np.linspace(0, 100, num_classes))

# Reshape data for LSTM input (assuming x_train and x_test are 3D tensors)
x_train_reshaped = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test_reshaped = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

# Define the LSTM model for classification
model = Sequential()
model.add(LSTM(units=50, input_shape=(x_train_reshaped.shape[1], 1)))
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model for classification
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

# Fit the model for classification
hist = model.fit(x_train_reshaped, y_train_classes, validation_split=0.2, epochs=3, batch_size=4096,
                 callbacks=[tf.keras.callbacks.ModelCheckpoint('model_lstm_classification.h5', monitor='val_loss', verbose=1, save_best_only=True),
                            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')])

# Evaluate the model on the test set using accuracy
test_results = model.evaluate(x_test_reshaped, y_test_classes)
print(f'Test Accuracy: {test_results[1]}')


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 50)                10400     
                                                                 
 dense_10 (Dense)            (None, 128)               6528      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 5)                 645       
                                                                 
Total params: 17573 (68.64 KB)
Trainable params: 17573 (68.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/3
76/76 [==============================] - ETA: 0s - loss: 0.1706 - accuracy: 0.9908
Epoch 1: val_loss improved from inf to 0.00088, saving mo

KeyboardInterrupt: 

#RNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

model = Sequential()
model.add(SimpleRNN(50, input_shape=(512,1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

# Train the model (Assuming X_train and y_train are your training data)
model.fit(x_train, y_train, epochs=10, batch_size=32)


#Temporal Convolutional Network (TCN)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten

model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(512,1)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

# Train the model (Assuming X_train and y_train are your training data)
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
#model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

#Gated Recurrent Units (GRUs)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

model = Sequential()
model.add(GRU(50, input_shape=(512,1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

# Train the model (Assuming X_train and y_train are your training data)
model.fit(x_train, y_train, epochs=10, batch_size=32)


#Transformer-based Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Transformer, Dense

model = Sequential()
model.add(Transformer(num_heads=2, d_model=512, ff_dim=256, input_shape=(512, 1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model (Assuming X_train and y_train are your training data)
model.fit(X_train, y_train, epochs=10, batch_size=32)


#CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, kernel_size=7, activation='relu', input_shape=(512, 1)))
model.add(Flatten())
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)

# Default

In [ ]:
import keras.models
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Input, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint




out = inp = Input(shape=(x_train.shape[1], 1))
for i in range(4):
    out = Conv1D(filters=32, kernel_size=7, padding='same')(out)
    out = Activation('relu')(out)
    out = MaxPooling1D(2, padding='same')(out)
out = GlobalMaxPooling1D()(out)
out = Dense(128)(out)
out = Dropout(0.2)(out)
out = Dense(1)(out)

model = Model(inputs=[inp], outputs=[out])
model.summary()
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
# fit model. the last 20% of the segments will be used for early stopping
hist = model.fit(x_train, y_train, validation_split=0.2, epochs=100, batch_size=4096,
                callbacks=[ModelCheckpoint(monitor='val_loss', filepath='model.x', verbose=1, save_best_only=True),
                            EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')])

# prediction
pred_test = model.predict(x_test).flatten()
for caseid in np.unique(c_test):
    case_mask = (c_test == caseid)
    pred_test[case_mask] = scipy.signal.medfilt(pred_test[case_mask], 15)

# calculate the performance
test_mae = np.mean(np.abs(y_test - pred_test))

In [ ]:
# pearson correlation coefficient
bis_corr = np.corrcoef(y_test, b_test)[0, 1]
our_corr = np.corrcoef(y_test, pred_test)[0, 1]

# scatter plot
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.scatter(y_test, b_test, s=1, alpha=0.5, c='violet', label=f'BIS ({bis_corr:4f})')
plt.xlim([0, 2])
plt.legend(loc="upper right")
plt.subplot(1, 2, 2)
plt.scatter(y_test, pred_test, s=1, alpha=0.5, c='orange', label=f'Ours ({our_corr:.4f})')
plt.xlim([0, 2])
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# plot for each case
for caseid in np.unique(c_test):
    case_mask = (c_test == caseid)
    case_len = np.sum(case_mask)
    if case_len == 0:
        continue

    our_mae = np.mean(np.abs(y_test[case_mask] - pred_test[case_mask]))
    print(f'Total MAE={test_mae:.4f}, CaseID {caseid}, MAE={our_mae:.4f}')

    t = np.arange(0, case_len)
    plt.figure(figsize=(20, 5))
    plt.plot(t, y_test[case_mask], label='MAC')
    plt.plot(t, pred_test[case_mask], label=f'Ours ({our_mae:.4f})')
    plt.legend(loc="upper left")
    plt.tight_layout()
    plt.xlim([0, case_len])
    plt.ylim([0, 2])
    plt.show()